In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pickle
import statistics
import community
import matplotlib as mlp
# mlp.use("Qt5Agg")
import matplotlib.pyplot as plt

import networkx as nx

import analysis as an
import pandas as pd
import statsmodels as st
import statsmodels.api as sm
from statsmodels.formula.api import ols

# from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
# from visbrain.io import download_file, read_stc

# from nilearn.connectome import ConnectivityMeasure

# from nilearn import plotting
#plt.matshow(rwd_mean_correlation_matrix,cmap='hot')

import numpy as np
import seaborn as sns

from scipy import stats as spy_stats
import scipy
import pprint as pp

In [2]:
basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/'

In [3]:
labels = pd.read_csv(os.path.join(basepath,'tmp','mod_labels.csv'), sep=',')

In [4]:
labels

,Index,label,Unnamed: 2,area,IC
0,73,R_Cerebellum,R_Cerebellum,Cerebellum,IC_73
1,59,R_Amygdala,R_Amygdala,Amygdala,IC_59
2,44,R_Auditory_18,R_A4_ROI,Auditory,IC_44
3,70,BrainStem,BrainStem,BrainStem,IC_70
4,79,BrainStem,BrainStem,BrainStem,IC_79
...,...,...,...,...,...
95,29,R_Visual_29,R_MST_ROI,Visual,IC_29
96,31,R_Visual_28,R_V3_ROI,Visual,IC_31
97,38,R_Visual_37,R_V2_ROI,Visual,IC_38
98,53,NaN,NaN,NaN,IC_53


In [5]:
labels.set_index('Index', inplace=True)

In [6]:
labels.head()

,label,Unnamed: 2,area,IC
Index,,,,
73,R_Cerebellum,R_Cerebellum,Cerebellum,IC_73
59,R_Amygdala,R_Amygdala,Amygdala,IC_59
44,R_Auditory_18,R_A4_ROI,Auditory,IC_44
70,BrainStem,BrainStem,BrainStem,IC_70
79,BrainStem,BrainStem,BrainStem,IC_79


In [7]:
note_dict={}
for i,j in labels.iterrows():
    print(i)
    print(j['area'])
    note_dict[i]=j['area']

73
Cerebellum
59
Amygdala
44
Auditory
70
BrainStem
79
BrainStem
80
BrainStem
85
BrainStem
90
BrainStem
93
BrainStem
94
BrainStem
97
BrainStem
62
Caudate
78
Caudate
41
Cerebellum
45
Cerebellum
48
Cerebellum
49
Cerebellum
50
Cerebellum
51
Cerebellum
52
Cerebellum
54
Cerebellum
55
Cerebellum
56
Cerebellum
57
Cerebellum
58
Cerebellum
61
Cerebellum
63
Cerebellum
65
Cerebellum
67
Cerebellum
69
Cerebellum
71
Cerebellum
72
Cerebellum
74
Cerebellum
76
Cerebellum
77
Cerebellum
81
Cerebellum
82
Cerebellum
83
Cerebellum
84
Cerebellum
87
Cerebellum
88
Cerebellum
89
Cerebellum
91
Cerebellum
92
Cerebellum
4
CinguloOperc
25
CinguloOperc
37
CinguloOperc
1
Default
5
Default
8
Default
9
Default
11
Default
21
Default
35
Default
13
DorsalAttn
17
DorsalAttn
22
DorsalAttn
33
DorsalAttn
6
FrontoParietal
23
FrontoParietal
32
FrontoParietal
39
FrontoParietal
43
FrontoParietal
47
FrontoParietal
14
MedialParietal
28
None
64
Putamen
68
Putamen
75
Putamen
26
Smhand
34
Smhand
42
Smhand
46
Smhand
20
Smmouth
40
Smmout

In [8]:
note_dict

{73: 'Cerebellum',
 59: 'Amygdala',
 44: 'Auditory',
 70: 'BrainStem',
 79: 'BrainStem',
 80: 'BrainStem',
 85: 'BrainStem',
 90: 'BrainStem',
 93: 'BrainStem',
 94: 'BrainStem',
 97: 'BrainStem',
 62: 'Caudate',
 78: 'Caudate',
 41: 'Cerebellum',
 45: 'Cerebellum',
 48: 'Cerebellum',
 49: 'Cerebellum',
 50: 'Cerebellum',
 51: 'Cerebellum',
 52: 'Cerebellum',
 54: 'Cerebellum',
 55: 'Cerebellum',
 56: 'Cerebellum',
 57: 'Cerebellum',
 58: 'Cerebellum',
 61: 'Cerebellum',
 63: 'Cerebellum',
 65: 'Cerebellum',
 67: 'Cerebellum',
 69: 'Cerebellum',
 71: 'Cerebellum',
 72: 'Cerebellum',
 74: 'Cerebellum',
 76: 'Cerebellum',
 77: 'Cerebellum',
 81: 'Cerebellum',
 82: 'Cerebellum',
 83: 'Cerebellum',
 84: 'Cerebellum',
 87: 'Cerebellum',
 88: 'Cerebellum',
 89: 'Cerebellum',
 91: 'Cerebellum',
 92: 'Cerebellum',
 4: 'CinguloOperc',
 25: 'CinguloOperc',
 37: 'CinguloOperc',
 1: 'Default',
 5: 'Default',
 8: 'Default',
 9: 'Default',
 11: 'Default',
 21: 'Default',
 35: 'Default',
 13: 'Dorsal

In [9]:
summary_dict=an.onetoughjar(os.path.join(basepath,'tmp','5_summary_dict_07-31-2020_12-26-08'))

In [10]:
summary_dict['NR']['ov']['modules'].keys()

dict_keys(['partition', 'values', 'graph', 'zdegree'])

In [11]:
for key, value in summary_dict['NR'].items():
    for subkey, subvalue in value.items():
#         print(subkey)
        if subkey == 'modules':
            print(subvalue.keys())
            summary_dict['NR'][key][subkey]['Q']=community.modularity(subvalue['partition'], subvalue['graph'], weight='weight')

dict_keys(['partition', 'values', 'graph', 'zdegree'])
dict_keys(['partition', 'values', 'graph', 'zdegree'])
dict_keys(['partition', 'values', 'graph', 'zdegree'])


In [12]:
print(summary_dict['NR']['no']['modules']['Q'])
print(summary_dict['NR']['ov']['modules']['Q'])
print(summary_dict['NR']['ob']['modules']['Q'])

0.48850785738519165
0.4901080895045965
0.4884168893617425


In [13]:
for key, value in summary_dict['NR'].items():
    for subkey, subvalue in value.items():
#         print(subkey)
        if subkey == 'modules':
            print(subvalue.keys())
            summary_dict['NR'][key][subkey]['dendrogram']=community.generate_dendrogram(subvalue['graph'],   weight='weight')

dict_keys(['partition', 'values', 'graph', 'zdegree', 'Q'])
dict_keys(['partition', 'values', 'graph', 'zdegree', 'Q'])
dict_keys(['partition', 'values', 'graph', 'zdegree', 'Q'])


In [14]:
x = summary_dict['NR']['ob']['modules']['dendrogram']

In [15]:
G=nx.erdos_renyi_graph(100, 0.01)
dendo = community.generate_dendrogram(G)

for level in range(len(x) - 1) :
    print("partition at level", level,"is", community.partition_at_level(dendo, level))


partition at level 0 is {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 4, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 14, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 25: 23, 26: 24, 27: 25, 28: 26, 29: 25, 30: 27, 31: 23, 32: 28, 33: 29, 34: 30, 35: 31, 36: 7, 37: 32, 38: 9, 39: 33, 40: 34, 41: 25, 42: 1, 43: 35, 44: 11, 45: 36, 46: 37, 47: 38, 48: 39, 49: 40, 50: 41, 51: 42, 52: 43, 53: 3, 54: 18, 55: 44, 56: 45, 57: 46, 58: 13, 59: 47, 60: 43, 61: 36, 62: 23, 63: 6, 64: 48, 65: 49, 66: 50, 67: 51, 68: 52, 69: 7, 70: 53, 71: 54, 72: 42, 73: 55, 74: 56, 75: 2, 76: 57, 77: 58, 78: 59, 79: 60, 80: 61, 81: 35, 82: 62, 83: 63, 84: 64, 85: 57, 86: 65, 87: 0, 88: 45, 89: 66, 90: 67, 91: 0, 92: 42, 93: 68, 94: 69, 95: 37, 96: 70, 97: 32, 98: 14, 99: 20}


In [17]:
#summary_dict=an.onetoughjar(os.path.join(basepath,'tmp','5_summary_dict_12-05-2019_03-29-19'))

summary_dict['NR']['no']['graphs'].nodes(data=True)

x=an.grace_graph(summary_dict['NR']['no']['graphs'],'Normal weight', thresh= 97, metric = 'centrality', position='spectral', basepath = basepath)

y=an.grace_graph(summary_dict['NR']['ov']['graphs'],  'Overweight',   thresh= 97, metric = 'centrality', position='spectral', basepath = basepath)

z=an.grace_graph(summary_dict['NR']['ob']['graphs'], 'Obese',  thresh= 97, metric = 'centrality', position='spectral',basepath = basepath)





0.41821221923076746
0.4129324982568799
0.41162024256410246


In [19]:
summary_dict['NR']['no']['graphs'].nodes(data=True)

NodeDataView({0: {'centrality': 0.0064897671177409555, 'clustering': 0.6437710437710438, 'PC': -0.8580148837645567, 'modules': 0}, 1: {'centrality': 0.002973716896117888, 'clustering': 0.7468599033816425, 'PC': -123.61026217098525, 'modules': 1}, 2: {'centrality': 0.004613266626234078, 'clustering': 0.6115384615384616, 'PC': -2.6821246893017743, 'modules': 2}, 3: {'centrality': 0.0021067150470575765, 'clustering': 0.7252252252252253, 'PC': -1621.55430050921, 'modules': 2}, 4: {'centrality': 0.005504181704497122, 'clustering': 0.6303191489361702, 'PC': -37.475017840862826, 'modules': 3}, 5: {'centrality': 0.005469547439511589, 'clustering': 0.581081081081081, 'PC': -6.721632253418668, 'modules': 4}, 6: {'centrality': 0.005565460193765314, 'clustering': 0.6334693877551021, 'PC': -1.6731865473609924, 'modules': 1}, 7: {'centrality': 0.0031554850913130426, 'clustering': 0.6693657219973009, 'PC': -39.20026783381694, 'modules': 2}, 8: {'centrality': 0.003099278857443236, 'clustering': 0.7323

In [20]:
# basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
demo_dict=an.onetoughjar(os.path.join(basepath,'tmp','demo_dict_11-13-2019_02-07-22'))


In [21]:
demo_dict['NR'].values()

dict_values([{'Age_in_Yrs': 24, 'ZygosityGT': ' ', 'Race': 'White', 'Ethnicity': 'Not Hispanic/Latino', 'BMI': 25.49, 'HbA1C': 5.4, 'Hypothyroidism': nan, 'Hyperthyroidism': nan, 'OtherEndocrn_Prob': nan, 'Mother_ID': 52259, 'Father_ID': 82122, 'ov_ob': 'ov'}, {'Age_in_Yrs': 27, 'ZygosityGT': ' ', 'Race': 'White', 'Ethnicity': 'Not Hispanic/Latino', 'BMI': 26.64, 'HbA1C': nan, 'Hypothyroidism': nan, 'Hyperthyroidism': nan, 'OtherEndocrn_Prob': nan, 'Mother_ID': 56037, 'Father_ID': 85858, 'ov_ob': 'ov'}, {'Age_in_Yrs': 35, 'ZygosityGT': ' ', 'Race': 'Black or African Am.', 'Ethnicity': 'Not Hispanic/Latino', 'BMI': 30.27, 'HbA1C': 6.0, 'Hypothyroidism': 0.0, 'Hyperthyroidism': 0.0, 'OtherEndocrn_Prob': 0.0, 'Mother_ID': 51283, 'Father_ID': 81149, 'ov_ob': 'ob'}, {'Age_in_Yrs': 22, 'ZygosityGT': ' ', 'Race': 'White', 'Ethnicity': 'Not Hispanic/Latino', 'BMI': 21.13, 'HbA1C': nan, 'Hypothyroidism': nan, 'Hyperthyroidism': nan, 'OtherEndocrn_Prob': nan, 'Mother_ID': 51969, 'Father_ID': 818

In [22]:
demos=pd.DataFrame.from_dict(demo_dict['NR'], orient='index')

In [ ]:
# demos.to_csv(os.path.join(basepath,'demos.csv'), sep=',')

In [23]:
print(z.sort_values(by=['degree'], ascending=False).head())
print(y.sort_values(by=['degree'], ascending=False).head())
print(x.sort_values(by=['degree'], ascending=False).head())

AttributeError: 'tuple' object has no attribute 'sort_values'

# Full graph metrics

In [ ]:
basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
#Load data from pickle if needed
# save_dict=an.onetoughjar(os.path.join(basepath,'tmp','save_dict_11-14-2019_02-46-01'))

In [ ]:
stat_dict={'MZ':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}},
           'DZ':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}},
           'NR':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}}}

In [ ]:
stat_dict={'MZ':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}},
           'DZ':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}},
           'NR':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}}}
for key, value in save_dict.items():
    for subkey,subval in value.items():
        for k,v in subval.items():
            if k in ['mean_FC','clustering_coeff','btn_centrality','PC']:
                stat_dict[key][k][subkey]=pd.DataFrame.from_dict(v, 'index')
                new_name=[]
                for item in list(stat_dict[key][k][subkey].columns[0:100]):
                    new_name.append('IC_%s'%(item+1))
                stat_dict[key][k][subkey].columns=new_name
                stat_dict[key][k][subkey]['group'] = subkey

for key, value in stat_dict.items():
    print(key)
    for k,v in value.items():
        print(k)
        df = pd.concat([v['no'],v['ov']], axis=0)
        df = pd.concat([df,v['ob']], axis=0)
        if k in ['clustering_coeff', 'btn_centrality', 'PC']:
            keys = []
            tables = []
            for variable in list(df.columns)[0:100]:
                m='%s ~ group'%variable
                model = ols(m, data=df).fit()
                anova_table = sm.stats.anova_lm(model, typ=2)

                keys.append(variable)
                tables.append(anova_table)

            df_anova = pd.concat(tables, keys=keys, axis=0)
            raw_p=df_anova['PR(>F)'].dropna()
            fdr=st.stats.multitest.fdrcorrection(raw_p, alpha=0.05, method='indep', is_sorted=False)
            stat_dict[key][k].update({'full':df, 'anova':df_anova, 'FDR':fdr})
        else:
            print(df)
            keys = []
            tables = []
            variable='IC_1'
            m='%s ~ group'%variable
            model = ols(m, data=df).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)

            keys.append(variable)
            tables.append(anova_table)

            df_anova = pd.concat(tables, keys=keys, axis=0)
            raw_p=df_anova['PR(>F)'].dropna()
            fdr=st.stats.multitest.fdrcorrection(raw_p, alpha=0.05, method='indep', is_sorted=False)
            stat_dict[key][k].update({'full':df, 'anova':df_anova, 'FDR':fdr})



In [ ]:
stat_dict['NR'].keys()

In [ ]:
for key, value in stat_dict.items():
    for subkey,subval in value.items():
        for k,v in subval.items():
            if k == 'FDR':
                s = set(v[0])
                if True in s:
                    print('there is a significant value here %s %s'%(key,subkey))
                    print(stat_dict[key][subkey]['FDR'][1])
                    print(stat_dict[key][subkey]['anova'])
                

In [ ]:
df=stat_dict['MZ']['btn_centrality']['full']
m='%s ~ group'%'IC_2'
model = ols(m, data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
model.summary()

Did not find any differences in the nodal metrics between groups in the not related condition  
IC_2 shows significantly different betweenness centrality between the obese and normal weight monozygotic twin subset. Probably due to a false discovery 

In [ ]:
# an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',stat_dict,'6_stat_dict')

In [ ]:
# stat_dict=an.onetoughjar(os.path.join(basepath,'tmp','6_stat_dict_11-20-2019_10-37-43'))


In [28]:
summary_dict['NR']['no'].keys()

dict_keys(['mean_FC', 'graphs', 'clustering_coeff', 'btn_centrality', 'PC', 'modules'])

In [29]:
no_edge_btw=nx.edge_betweenness_centrality(summary_dict['NR']['no']['graphs'], normalized=True, weight='weight')

In [30]:
no_edge_btw

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.7142424242424242,
 (0, 4): 0.0,
 (0, 5): 0.0,
 (0, 6): 0.0,
 (0, 7): 0.0,
 (0, 8): 0.0,
 (0, 9): 0.0,
 (0, 10): 0.009898989898989899,
 (0, 11): 0.0,
 (0, 12): 0.0,
 (0, 13): 0.0,
 (0, 14): 0.0,
 (0, 15): 0.005858585858585858,
 (0, 16): 0.0,
 (0, 17): 0.0,
 (0, 18): 0.0,
 (0, 19): 0.0,
 (0, 20): 0.0,
 (0, 21): 0.0,
 (0, 22): 0.0,
 (0, 23): 0.0,
 (0, 24): 0.0,
 (0, 25): 0.0,
 (0, 26): 0.0,
 (0, 27): 0.0,
 (0, 28): 0.0,
 (0, 29): 0.1304040404040404,
 (0, 30): 0.0,
 (0, 31): 0.0,
 (0, 32): 0.0,
 (0, 33): 0.5517171717171717,
 (0, 34): 0.0,
 (0, 35): 0.0,
 (0, 36): 0.0,
 (0, 37): 0.0,
 (0, 38): 0.0,
 (0, 39): 0.0,
 (0, 40): 0.0,
 (0, 41): 0.0,
 (0, 42): 0.0,
 (0, 43): 0.0,
 (0, 44): 0.0,
 (0, 45): 0.0,
 (0, 46): 0.0,
 (0, 47): 0.0,
 (0, 48): 0.0,
 (0, 49): 0.0,
 (0, 50): 0.0,
 (0, 51): 0.0,
 (0, 52): 0.0,
 (0, 53): 0.0,
 (0, 54): 0.0,
 (0, 55): 0.0,
 (0, 56): 0.0,
 (0, 57): 0.0,
 (0, 58): 0.0,
 (0, 59): 0.0,
 (0, 60): 0.0,
 (0, 61): 0.0,
 

In [31]:
ov_edge_btw=nx.edge_betweenness_centrality(summary_dict['NR']['ov']['graphs'], normalized=True, weight='weight')

In [32]:
ob_edge_btw=nx.edge_betweenness_centrality(summary_dict['NR']['ob']['graphs'], normalized=True, weight='weight')

In [33]:
summary_dict['NR']['ob']['edge_btw']=ob_edge_btw
summary_dict['NR']['ov']['edge_btw']=ov_edge_btw
summary_dict['NR']['no']['edge_btw']=no_edge_btw

In [34]:
nx.set_edge_attributes(summary_dict['NR']['ob']['graphs'], ob_edge_btw, 'betweenness')
nx.set_edge_attributes(summary_dict['NR']['ov']['graphs'], ov_edge_btw, 'betweenness')
nx.set_edge_attributes(summary_dict['NR']['no']['graphs'], no_edge_btw, 'betweenness')

In [35]:
an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',summary_dict,'5_summary_dict')

In [39]:
def grace_graph(graph, group,  **kwargs):
    e,w = zip(*nx.get_edge_attributes(graph, 'weight').items())
    if bool(kwargs) == False:
        positions = nx.circular_layout(graph)
        size = 100
        title= "Modularity and edge weights \n of average %s graph"%(group)
        save="%s_graph.png"%(group)
    else:
        if 'edges' in kwargs:
            print('this is a betweenness graph')
            edges, weights = zip(*nx.get_edge_attributes(graph, 'betweenness').items())
        else:
            edges, weights = zip(*nx.get_edge_attributes(graph, 'weight').items())
        if 'position' in kwargs and kwargs['position']=='spectral':
            positions = nx.spectral_layout(graph)
            title= "Spectral modularity and edge weights \n of average %s graph"%(group)
            save="Spectral_%s_graph.png"%(group)
        elif 'position' in kwargs and kwargs['position']=='spring':
            positions = nx.spring_layout(graph)
            title= "Spring modularity and edge weights \n of average %s graph"%(group)
            save="Sping_%s_graph.png"%(group)
        else:
            positions = nx.circular_layout(graph)
            title= "Circle modularity and edge weights \n of average %s graph"%(group)
            save="Circle_%s_graph.png"%(group)
        if 'metric' in kwargs:
            nodes, size = zip(*nx.get_node_attributes(graph, kwargs['metric']).items())
        else:
            size = 100
            title = "basic"
            save="%s_graph.png"%(group)
        if 'thresh' in kwargs:
            tile=kwargs['thresh']
            purr=np.percentile(w, tile)
            print(purr)
            graph=an.threshold2(graph,purr)
        
    nodes, color = zip(*nx.get_node_attributes(graph, 'modules').items()) #if your modules are named different change here
    # nodes, names = zip(*nx.get_node_attributes(graph, 'label').items()) #if your modules are named different change here
    g=graph
    #Figure size
    plt.figure(figsize=(80,50))

    #draws nodes
    color = np.array(color)
    n_color=len(list(set(color)))
    # nColormap=plt.cm.Set3 #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    # get discrete colormap
    nColormap = plt.get_cmap('Set3', n_color)

    # scaling
    sz=np.array(size)
    scale=15000/sz.max()
    sza=sz*scale
    # print(sz.shape)

    y=nx.draw_networkx_nodes(g,positions,
                           node_color=color,
                           node_size=sza,
                           alpha=0.8,
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(g, positions,
                            # labels = label_dict,
                            font_size=50,
                            font_family='sans-serif',
                            fontweight = 'bold')

    #draw edges
#     print(weights)
    weights=np.array(weights)
    eColormap=plt.cm.gist_rainbow #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    # scaling
    wt=list(set(weights))
    wt=np.array(wt)
    wt2=-np.sort(-wt)
    wt0=wt2[1]

    escale=1/wt0
    esza=weights*escale
    E=list(set(esza))
    E2=-np.sort(-np.array(E))
    M=E2[1]
    m=esza.min()

    x=nx.draw_networkx_edges(g, positions,
                           edge_list=edges,
                           style='solid',
                           width = np.square(esza)*5,
                           edge_color = esza,
                           edge_cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)

    #COLORBAR STUFF
    node_bar=plt.colorbar(y, label='Module value')

    tick_locs = (np.arange(n_color) + 0.5)*(n_color-1)/n_color
    node_bar.set_ticks(tick_locs)

    # set tick labels (as before)
    node_bar.set_ticklabels(np.arange(n_color))


    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = m, vmax=M))
    sm._A = []
    edge_bar=plt.colorbar(sm)

    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
        l.set_verticalalignment('center')

    node_bar.set_label('Modularity',fontsize = 50)
    edge_bar.set_label('Strength of edge weight',fontsize = 50)
    # Final plot stuff
    plt.axis('off')

    plt.title(title, fontsize = 100)
    basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/images'

    plt.savefig(os.path.join(basepath,save), format="PNG")
    plt.show()
    return()

In [40]:
summary_dict['NR']['ob']['graphs'].nodes(data=True)

NodeDataView({0: {'centrality': 0.007252218675881815, 'clustering': 0.6016326530612245, 'PC': -0.23723986224352767, 'modules': 0}, 1: {'centrality': 0.0034956349708382935, 'clustering': 0.7082125603864734, 'PC': -112.93234849455159, 'modules': 1}, 2: {'centrality': 0.003598121226806917, 'clustering': 0.6410256410256411, 'PC': -2.162166474246793, 'modules': 2}, 3: {'centrality': 0.002542525041040711, 'clustering': 0.7044534412955465, 'PC': -81.67892894399134, 'modules': 2}, 4: {'centrality': 0.006065641468451796, 'clustering': 0.5966312056737588, 'PC': -67.33406015524196, 'modules': 3}, 5: {'centrality': 0.005483975994560874, 'clustering': 0.5512605042016807, 'PC': -5.85744568969122, 'modules': 1}, 6: {'centrality': 0.004674963903752665, 'clustering': 0.6345975948196114, 'PC': -4.063857866257118, 'modules': 4}, 7: {'centrality': 0.003243890688635134, 'clustering': 0.6693657219973009, 'PC': -17.95647537816139, 'modules': 2}, 8: {'centrality': 0.005709497529550663, 'clustering': 0.6556862

In [41]:
grace_graph(summary_dict['NR']['ob']['graphs'], 'Obese',  thresh= 0, metric = 'centrality', position='circle', edges=True)

grace_graph(summary_dict['NR']['ov']['graphs'], 'Overweight',  thresh= 0, metric = 'centrality', position='circle', edges=True)

grace_graph(summary_dict['NR']['no']['graphs'], 'Normal weight',  thresh= 0, metric = 'centrality', position='circle', edges=True)



this is a betweenness graph
-0.5011753589743589
this is a betweenness graph
-0.490307495412844
this is a betweenness graph
-0.4972256025641026


()

## Checking functional connectivity 

In [ ]:
def plot_matrices(matrices, matrix_kind):
    n_matrices = len(matrices)
    fig = plt.figure(figsize=(n_matrices * 4, 4))
    for n_subject, matrix in enumerate(matrices):
        plt.subplot(1, n_matrices, n_subject + 1)
        matrix = matrix.copy()  # avoid side effects
        # Set diagonal to zero, for better visualization
        np.fill_diagonal(matrix, 0)
        vmax = np.max(np.abs(matrix))
        title = '{0}, subject {1}'.format(matrix_kind, n_subject)
        plotting.plot_matrix(matrix, vmin=-vmax, vmax=vmax, cmap='RdBu_r',
                             title=title, figure=fig, colorbar=False)
        


In [ ]:
basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
file_dict=an.onetoughjar(os.path.join(basepath,'tmp','file_dict_11-14-2019_10-07-24'))


In [ ]:
file_dict['NR']['ov']['134223']

In [ ]:
listcorr_dict={'MZ':{'no':[],'ov':[],'ob':[]},'DZ':{'no':[],'ov':[],'ob':[]},'NR':{'no':[],'ov':[],'ob':[]}}
for key, value in file_dict.items():
    for subkey, subval in value.items():
        print(subkey)
        for k,v in subval.items():
            listcorr_dict[key][subkey].append(v.to_numpy())

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation')

for key, value in listcorr_dict['NR'].items():
    correlation_matrices = correlation_measure.fit_transform(value)
    # All individual coefficients are stacked in a unique 2D matrix.
    print('Correlations of %s subjects are stacked in an array of shape {0}'%key
          .format(correlation_matrices.shape))
    mean_correlation_matrix = correlation_measure.mean_
    print('Mean correlation has shape {0}.'.format(mean_correlation_matrix.shape))


    plt.figure(figsize=(40,25))
    ax = sns.heatmap(mean_correlation_matrix, linewidth=0.5, cmap='coolwarm')

    #Save the figure
    # ax.figure.savefig("/Users/jennygilbert/Documents/betaseries_bevel/reward_coor_mat.png")

    #plot_matrices(rwd_correlation_matrices[:5], 'reward')
    #plot_matrices(rwd_mean_correlation_matrix, 'reward')



    plt.show()

In [ ]:
def corrector(x, alpha):
    results=x[1].ravel()
    mask = np.isfinite(results)
    pval_corrected = np.empty(results.shape)
    pval_corrected.fill(np.nan)
    pval_corrected[mask] = st.stats.multitest.multipletests(results[mask],alpha=alpha,method='fdr_bh')[1]
    p=np.reshape(pval_corrected, (100, 100))
    print(np.nanmin(p))
    ps = 1-p
    print(np.nanmax(ps))
    coor_fig(ps)
    return(p)

In [ ]:
def coor_fig(df):
    plt.figure(figsize=(40,25))
    m=np.nanmin(df)
    M=np.nanmax(df)
    print('The max p value is %f'%M)
    sns.heatmap(df, linewidth=0.5, 
                vmin=m, vmax=1, 
                cmap=sns.cubehelix_palette(10000), 
                cbar_kws={'ticks': [0.0, 0.2, 0.4, 0.5, 0.7, 0.8,0.975 ,1.0]})
    return(plt.show())

In [ ]:
a=listcorr_dict['NR']['no']
b=listcorr_dict['NR']['ov']
c=listcorr_dict['NR']['ob']
test=spy_stats.f_oneway(a, b, c)
anova_results=corrector(test, 0.05)

### Between no and ov & no and ob
p=0.05/2  
p=0.025

In [ ]:
no_ov=spy_stats.ttest_ind(a, b, nan_policy='omit')
no_ov_results=corrector(no_ov, 0.025)

In [ ]:
no_ob=spy_stats.ttest_ind(a, c, nan_policy='omit')
no_ob_results=corrector(no_ob, 0.025)

## Summary
There appears to be no differences in the graph metrics between nodes. Nor a difference in functional connectivity

# Between modules metrics

In [42]:
# basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
# #Load data from pickle if needed
# summary_dict=an.onetoughjar(os.path.join(basepath,'tmp','summary_dict_11-14-2019_04-33-33'))
for key, value in summary_dict.items():
    for k,v in value.items():
        # community.induced_graph(partition dictionary, graph)
         comm_graph = community.induced_graph(v['modules']['partition'], v['graphs'])
         v.update(comm_graph = comm_graph)

In [43]:
summary_dict['NR']['no']['comm_graph'].nodes(data=True)

NodeDataView({0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}})

In [44]:
btw_no=nx.edge_betweenness_centrality(summary_dict['NR']['no']['comm_graph'], weight=True,  normalized = True)
btw_no

{(0, 0): 0.0,
 (0, 1): 0.03571428571428571,
 (0, 2): 0.03571428571428571,
 (0, 3): 0.03571428571428571,
 (0, 4): 0.03571428571428571,
 (0, 5): 0.03571428571428571,
 (0, 6): 0.03571428571428571,
 (0, 7): 0.03571428571428571,
 (1, 1): 0.0,
 (1, 2): 0.03571428571428571,
 (1, 3): 0.03571428571428571,
 (1, 4): 0.03571428571428571,
 (1, 5): 0.03571428571428571,
 (1, 6): 0.03571428571428571,
 (1, 7): 0.03571428571428571,
 (2, 2): 0.0,
 (2, 3): 0.03571428571428571,
 (2, 4): 0.03571428571428571,
 (2, 5): 0.03571428571428571,
 (2, 6): 0.03571428571428571,
 (2, 7): 0.03571428571428571,
 (3, 3): 0.0,
 (3, 4): 0.03571428571428571,
 (3, 5): 0.03571428571428571,
 (3, 6): 0.03571428571428571,
 (3, 7): 0.03571428571428571,
 (4, 4): 0.0,
 (4, 5): 0.03571428571428571,
 (4, 6): 0.03571428571428571,
 (4, 7): 0.03571428571428571,
 (5, 5): 0.0,
 (5, 6): 0.03571428571428571,
 (5, 7): 0.03571428571428571,
 (6, 6): 0.0,
 (6, 7): 0.03571428571428571,
 (7, 7): 0.0}

In [45]:
btw_ov=nx.edge_betweenness_centrality(summary_dict['NR']['ov']['comm_graph'], weight=True, normalized = True)
btw_ov

{(0, 0): 0.0,
 (0, 1): 0.03571428571428571,
 (0, 2): 0.03571428571428571,
 (0, 3): 0.03571428571428571,
 (0, 4): 0.03571428571428571,
 (0, 5): 0.03571428571428571,
 (0, 6): 0.03571428571428571,
 (0, 7): 0.03571428571428571,
 (1, 1): 0.0,
 (1, 2): 0.03571428571428571,
 (1, 3): 0.03571428571428571,
 (1, 4): 0.03571428571428571,
 (1, 5): 0.03571428571428571,
 (1, 6): 0.03571428571428571,
 (1, 7): 0.03571428571428571,
 (2, 2): 0.0,
 (2, 3): 0.03571428571428571,
 (2, 4): 0.03571428571428571,
 (2, 5): 0.03571428571428571,
 (2, 6): 0.03571428571428571,
 (2, 7): 0.03571428571428571,
 (3, 3): 0.0,
 (3, 4): 0.03571428571428571,
 (3, 5): 0.03571428571428571,
 (3, 6): 0.03571428571428571,
 (3, 7): 0.03571428571428571,
 (4, 4): 0.0,
 (4, 5): 0.03571428571428571,
 (4, 6): 0.03571428571428571,
 (4, 7): 0.03571428571428571,
 (5, 5): 0.0,
 (5, 6): 0.03571428571428571,
 (5, 7): 0.03571428571428571,
 (6, 6): 0.0,
 (6, 7): 0.03571428571428571,
 (7, 7): 0.0}

In [46]:
btw_ob=nx.edge_betweenness_centrality(summary_dict['NR']['ob']['comm_graph'], weight=True, normalized = True)
btw_ob

{(0, 0): 0.0,
 (0, 1): 0.022222222222222223,
 (0, 2): 0.022222222222222223,
 (0, 3): 0.022222222222222223,
 (0, 4): 0.022222222222222223,
 (0, 5): 0.022222222222222223,
 (0, 6): 0.022222222222222223,
 (0, 7): 0.022222222222222223,
 (0, 8): 0.022222222222222223,
 (0, 9): 0.022222222222222223,
 (1, 1): 0.0,
 (1, 2): 0.022222222222222223,
 (1, 3): 0.022222222222222223,
 (1, 4): 0.022222222222222223,
 (1, 5): 0.022222222222222223,
 (1, 6): 0.022222222222222223,
 (1, 7): 0.022222222222222223,
 (1, 8): 0.022222222222222223,
 (1, 9): 0.022222222222222223,
 (2, 2): 0.0,
 (2, 3): 0.022222222222222223,
 (2, 4): 0.022222222222222223,
 (2, 5): 0.022222222222222223,
 (2, 6): 0.022222222222222223,
 (2, 7): 0.022222222222222223,
 (2, 8): 0.022222222222222223,
 (2, 9): 0.022222222222222223,
 (3, 3): 0.0,
 (3, 4): 0.022222222222222223,
 (3, 5): 0.022222222222222223,
 (3, 6): 0.022222222222222223,
 (3, 7): 0.022222222222222223,
 (3, 8): 0.022222222222222223,
 (3, 9): 0.022222222222222223,
 (4, 4): 0.0,


In [ ]:
def corr_mod(mat):
    plt.figure(figsize=(5,5))
    ax = sns.heatmap(mat, linewidth=0.5, cmap='coolwarm')
    return(plt.show())

In [47]:
A = nx.to_numpy_matrix(summary_dict['NR']['no']['comm_graph'])
B = nx.to_numpy_matrix(summary_dict['NR']['ov']['comm_graph'])
C = nx.to_numpy_matrix(summary_dict['NR']['ob']['comm_graph'])

In [48]:
nx.to_pandas_adjacency(summary_dict['NR']['no']['comm_graph'])

,0,1,2,3,4,5,6,7
0,6.839178,2.361006,-0.502959,-0.421221,0.760298,0.793000,-0.056822,0.153394
1,2.361006,76.266018,-82.851796,-5.192455,5.932729,3.131386,0.276001,3.194532
2,-0.502959,-82.851796,82.860900,-6.092118,-11.600036,-13.668109,-0.778605,-10.902000
3,-0.421221,-5.192455,-6.092118,38.501566,-11.812784,5.099932,0.281118,0.997265
4,0.760298,5.932729,-11.600036,-11.812784,10.655279,-0.549975,-0.177443,0.615818
5,0.793000,3.131386,-13.668109,5.099932,-0.549975,9.833891,-0.035917,1.062549
6,-0.056822,0.276001,-0.778605,0.281118,-0.177443,-0.035917,1.000000,0.078132
7,0.153394,3.194532,-10.902000,0.997265,0.615818,1.062549,0.078132,8.877413


In [49]:
corr_mod(A)
corr_mod(B)
corr_mod(C)

NameError: name 'corr_mod' is not defined

In [50]:
def module_fig(G, Type):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #nodes, size = zip(*nx.get_node_attributes(G,'clustering').items())

    positions=nx.circular_layout(G)
    plt.figure(figsize=(80,50))
    ### NODES ####
    color = np.array(list(G.nodes))
    color = np.array(color)
    n_color=len(list(set(color)))
    nColormap = plt.get_cmap('Set3', n_color)
    cM=color.max()
    cm=color.min()
    y=nx.draw_networkx_nodes(G,positions,
                           node_color=color,
                           node_size=15000,
                           alpha=1.0,
                           cmap= nColormap,
                           vmin=cm,vmax=cM )

    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=100,
                            font_family='sans-serif', fontweight = 'bold')
    
    ### EDGES ####
    weights=np.array(weights)
    # scaling
    wt=list(set(weights))
    wt=np.array(wt)
    wt2=-np.sort(-wt)
    wt0=wt2[1]

    escale=1/wt0
    esza=weights*escale
    E=list(set(esza))
    E2=-np.sort(-np.array(E))
    M=E2[1]
    m=esza.min()
    
    
    eColormap=plt.cm.gist_rainbow
    
    x=nx.draw_networkx_edges(G, positions, 
                             edge_list=edges,
                             style='solid', 
                             width = np.square(esza*5),
                             edge_color = weights, 
                             edge_vmin=m, 
                             edge_vmax=M, 
                             edge_cmap= eColormap)

    
    node_bar=plt.colorbar(y, label='Module value')
    
    tick_locs = (np.arange(n_color) + 0.5)*(n_color-1)/n_color
    node_bar.set_ticks(tick_locs)

    # set tick labels (as before)
    node_bar.set_ticklabels(np.arange(n_color))


    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = m, vmax=M))
    sm._A = []
    edge_bar=plt.colorbar(sm)

    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
        l.set_verticalalignment('center')

    node_bar.set_label('Modularity',fontsize = 50)
    edge_bar.set_label('Strength of edge weight',fontsize = 50)
    
    plt.title("Module Connectivity Weights %s"%Type, fontsize = 100)
    plt.axis('off')
    basepath='/Users/gracer/Google Drive/HCP_graph/1200/images'
    plt.savefig(os.path.join(basepath,"modularity_%s.png"%(Type)), format="PNG")
    # plt.show()

In [53]:
basepath

'/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/'

In [52]:
an.module_fig(summary_dict['NR']['no']['comm_graph'], 'Normal', basepath = basepath)
an.module_fig(summary_dict['NR']['ov']['comm_graph'], 'Overweight',basepath = basepath)
an.module_fig(summary_dict['NR']['ob']['comm_graph'], 'Obese',basepath = basepath)

## Creating subgraphs per module

In [54]:
unique, counts = np.unique(summary_dict['NR']['no']['modules']['values'], return_counts=True)
print(np.asarray((unique, counts)).T)

[[ 0  4]
 [ 1 27]
 [ 2 26]
 [ 3 20]
 [ 4  7]
 [ 5  7]
 [ 6  1]
 [ 7  8]]


In [55]:
mod_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }

In [56]:
for key, values in summary_dict.items():
    print(key)
    for k,v in values.items():
        print(k)
        unique, counts = np.unique(summary_dict[key][k]['modules']['values'], return_counts=True)
        for i in unique:
            mod_dict[key][k].update({i:[]})
        for q, w in summary_dict[key][k]['modules']['partition'].items():
            mod_dict[key][k][w].append(q)

MZ
no
ov
ob
DZ
no
ov
ob
NR
no
ov
ob


In [110]:
mod_dict['NR']['no']

{0: [0, 24, 31, 38],
 1: [1,
  6,
  8,
  11,
  13,
  21,
  23,
  28,
  32,
  36,
  39,
  41,
  43,
  45,
  47,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  58,
  60,
  65,
  73],
 2: [2,
  3,
  7,
  10,
  12,
  15,
  16,
  17,
  18,
  19,
  20,
  22,
  26,
  27,
  29,
  33,
  34,
  37,
  40,
  42,
  44,
  46,
  80,
  87,
  95,
  96],
 3: [4,
  25,
  30,
  48,
  57,
  61,
  67,
  69,
  71,
  74,
  76,
  77,
  81,
  82,
  83,
  84,
  88,
  89,
  91,
  92],
 4: [5, 9, 14, 35, 59, 63, 72],
 5: [62, 64, 66, 68, 75, 78, 86],
 6: [70],
 7: [79, 85, 90, 93, 94, 97, 98, 99]}

In [58]:
summary_dict['NR']['no'].keys()

dict_keys(['mean_FC', 'graphs', 'clustering_coeff', 'btn_centrality', 'PC', 'modules', 'edge_btw', 'comm_graph'])

In [59]:
subgraph_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }
for key, value in mod_dict.items():
    print(key)
    for k, v in value.items():
        print(k)
        G=summary_dict[key][k]['graphs']
        for q,w in v.items():
            print(q)
            print(w)
#             H=nx.subgraph_view(G,filter_node=w)
            H = G.subgraph(w).copy()
            subgraph_dict[key][k][q]=H
            
    

MZ
no
0
[0, 24, 31, 38]
1
[1, 5, 6, 8, 9, 11, 13, 14, 21, 23, 32, 35, 39, 41, 43, 45, 47, 49, 50, 51, 52, 53, 55, 56, 58, 60, 63, 65, 72]
2
[2, 3, 7, 10, 12, 15, 16, 18, 19, 20, 22, 26, 27, 28, 29, 33, 34, 36, 37, 40, 42, 44, 46, 59, 95]
3
[4, 17, 25, 30, 48, 54, 57, 61, 67, 69, 71, 73, 74, 76, 77, 81, 82, 83, 84, 87, 88, 89, 91, 92]
4
[62, 64, 66, 68, 75, 78, 86]
5
[70]
6
[79, 85, 90, 93, 94, 97, 98, 99]
7
[80, 96]
ov
0
[0, 24, 31, 38]
1
[1, 6, 8, 11, 13, 21, 23, 28, 32, 36, 39, 41, 43, 45, 47, 49, 50, 51, 52, 53, 55, 56, 58, 60, 65]
2
[2, 3, 7, 10, 12, 15, 16, 18, 19, 20, 22, 26, 27, 29, 33, 34, 37, 40, 42, 44, 46, 80, 87, 95, 96]
3
[4, 17, 25, 30, 48, 54, 57, 61, 67, 69, 71, 73, 74, 76, 77, 81, 82, 83, 84, 88, 89, 91, 92, 93]
4
[5, 9, 14, 35, 59, 63, 72]
5
[62, 64, 66, 68, 75, 78, 86]
6
[70]
7
[79, 85, 90, 94, 97, 98, 99]
ob
0
[0, 24, 31, 38]
1
[1, 5, 6, 8, 9, 11, 13, 14, 21, 23, 32, 35, 39, 41, 43, 45, 47, 49, 50, 51, 52, 53, 55, 56, 60, 65, 72]
2
[2, 3, 7, 10, 12, 15, 16, 17, 18, 

In [60]:
subgraph_dict['NR']

{'no': {0: <networkx.classes.graph.Graph at 0x1a373f8198>,
  1: <networkx.classes.graph.Graph at 0x1a373f8208>,
  2: <networkx.classes.graph.Graph at 0x1a373f82b0>,
  3: <networkx.classes.graph.Graph at 0x1a373f82e8>,
  4: <networkx.classes.graph.Graph at 0x1a373f8358>,
  5: <networkx.classes.graph.Graph at 0x1a373f8400>,
  6: <networkx.classes.graph.Graph at 0x1a373f84a8>,
  7: <networkx.classes.graph.Graph at 0x1a373f8518>},
 'ov': {0: <networkx.classes.graph.Graph at 0x1a373f8550>,
  1: <networkx.classes.graph.Graph at 0x1a373f85c0>,
  2: <networkx.classes.graph.Graph at 0x1a373f8630>,
  3: <networkx.classes.graph.Graph at 0x1a373f86d8>,
  4: <networkx.classes.graph.Graph at 0x1a373f8748>,
  5: <networkx.classes.graph.Graph at 0x1a373f87b8>,
  6: <networkx.classes.graph.Graph at 0x1a373f8828>,
  7: <networkx.classes.graph.Graph at 0x1a373f88d0>},
 'ob': {0: <networkx.classes.graph.Graph at 0x1a373f8940>,
  1: <networkx.classes.graph.Graph at 0x1a373f8978>,
  2: <networkx.classes.gra

In [61]:
subgraph_dict['NR']['no'][6].nodes(data=True)

NodeDataView({70: {'centrality': 0.0026298687658100153, 'clustering': 0.7437557816836263, 'PC': -4.243771076818776, 'modules': 6}})

In [62]:
def threshold(G, min_correlation):
    ##Creates a copy of the graph
    H = G.copy()
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0
#         print(min_correlation)
        if weight["weight"] < min_correlation:
            H.remove_edge(stock1, stock2)
    return(H)

In [63]:
#subgraph_dict=an.onetoughjar(os.path.join(basepath,'tmp','7_subgraph_dict_11-20-2019_10-40-04'))


In [67]:
def inner_mod(graph, metric, group, tile, style):
    e,w = zip(*nx.get_edge_attributes(graph, 'weight').items())
    purr=np.percentile(w, tile)
    print(purr)
    g=an.threshold2(graph,purr)

    edges,weights = zip(*nx.get_edge_attributes(g, 'weight').items())
    weights=np.array(weights)
    print(weights.min())
    
    nodes, color = zip(*nx.get_node_attributes(g, metric).items()) 
    nodes, size = zip(*nx.get_node_attributes(g, metric).items())
    nodes, positions = zip(*nx.get_node_attributes(g,'modules').items())
    #positions
    if style == 'spectral':
        positions=nx.spectral_layout(g) #this is defining a circluar graph, if you want a different one you change the circular part of this line
    elif style == 'spring':
        positions=nx.spring_layout(g)
    else:
        positions=nx.circular_layout(g)
    #Figure size
    plt.figure(figsize=(80,50))

    #draws nodes
    color = np.array(color)
    colz=scipy.stats.zscore(color)
    nColormap=plt.cm.cool #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=colz.max()
    cm=colz.min()
    

    scale=15000/colz.max()
    y=nx.draw_networkx_nodes(g,positions,
                           node_color=colz,
                           node_size=np.square(colz)*scale,
                           alpha=0.8,
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    keeps=g.nodes()
    dict_you_want = { your_key: note_dict[your_key] for your_key in keeps }
    nx.draw_networkx_labels(g, positions, font_size=50,
                            labels=dict_you_want,
                            font_family='sans-serif', 
                            fontweight = 'bold')

    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.gist_rainbow #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    # scaling
    wt=list(set(weights))
    wt=np.array(wt)
    wt2=-np.sort(-wt)
    wt0=wt2[1]

    escale=1/wt0
    esza=weights*escale
    E=list(set(esza))
    E2=-np.sort(-np.array(E))
    M=E2[1]
    m=esza.min()

    x=nx.draw_networkx_edges(g, positions,
                           edge_list=edges,
                           style='solid',
                           width = np.square(esza)*5,
                           edge_color = esza,
                           edge_cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)

    #format the colorbar
    node_bar=plt.colorbar(y, label='Module value')

    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = m, vmax=M))
    sm._A = []
    edge_bar=plt.colorbar(sm)


    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
        
    node_bar.set_label('%s'%metric,fontsize = 50)
    edge_bar.set_label('Strength of edge weight',fontsize = 50)

    plt.axis('off')
    plt.title("%s and edge weights of \n average %s graph"%(metric, group), fontsize = 100)
    basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/images'

    plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric,style,group)), format="PNG")
    plt.show()


In [111]:
subgraph_dict['NR']['ov']

{0: <networkx.classes.graph.Graph at 0x1a3cfa4550>,
 1: <networkx.classes.graph.Graph at 0x1a3cfa4160>,
 2: <networkx.classes.graph.Graph at 0x1a3cfa4668>,
 3: <networkx.classes.graph.Graph at 0x1a3cfa4470>,
 4: <networkx.classes.graph.Graph at 0x1a3cfa4588>,
 5: <networkx.classes.graph.Graph at 0x1a3cfa4390>,
 6: <networkx.classes.graph.Graph at 0x1a3cfa40b8>,
 7: <networkx.classes.graph.Graph at 0x1a3cfa4048>,
 8: <networkx.classes.graph.Graph at 0x1a3cfa4400>,
 9: <networkx.classes.graph.Graph at 0x1a3cfa4278>}

In [69]:
inner_mod(subgraph_dict['NR']['ov'][8], 'centrality', '%s module %s'%('ov','8'), 40, 'circle')

0.10601479816513762
0.10601479816513762


In [70]:
interest = [1,2,4,5,6,8]
for i in interest:
    for key, value in subgraph_dict['NR'].items():    
        print(key)
        inner_mod(value[i], 'centrality', '%s module %s'%(key,i), 40, 'circle')

no
0.08502862179487179
0.08502862179487179
ov
0.15421511926605502
0.15421511926605502
ob
0.17226933589743593
0.17251388461538464
no
0.11339195384615386
0.11345825641025642
ov
0.11198481100917432
0.11224785321100918
ob
0.12058560256410261
0.12157952564102563
no
0.09786729102564103
0.09797682692307692
ov
0.40735336513761466
0.4183935779816514
ob
0.13810708974358976
0.13810708974358976


/Users/gracer/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


no
0.07152491923076924
0.07284040384615385
ov
0.0966071908256881
0.10102650458715598
ob
0.2920741794871795
0.29691896153846153
no
0.09275233333333335
0.09275233333333335
ov
0.17262182568807338
0.17262182568807338
ob
0.11403446666666667
0.11454161538461538
no


KeyError: 8

In [71]:
#an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',subgraph_dict,'7_subgraph_dict')

already have tmp


In [112]:
subgraph_dict['NR']['no'][0].nodes(data=True)

NodeDataView({0: {'centrality': 0.0064897671177409555, 'clustering': 0.6437710437710438, 'PC': -1.0326969056632502, 'modules': 0}, 24: {'centrality': 0.01252232649388041, 'clustering': 0.6076388888888888, 'PC': 0.47330230071858304, 'modules': 0}, 38: {'centrality': 0.0065647443593222984, 'clustering': 0.5475609756097561, 'PC': 0.16462839848007893, 'modules': 0}, 31: {'centrality': 0.011105955087203292, 'clustering': 0.6101694915254238, 'PC': 0.5433342508090113, 'modules': 0}})

In [73]:
modstat_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }

In [74]:
for key, value in subgraph_dict.items():
    print(key)
    for subkey,subval in value.items():
        print(subkey)
        liist=list(subgraph_dict[key][subkey].keys())
        for i in liist:
            modstat_dict[key][subkey].update({i:[]})
        for k,v in subval.items():
            print(k)
            print(subkey)
#             print(dict(v.nodes(data=True)))
            modstat_dict[key][subkey][k]=pd.DataFrame.from_dict(dict(subgraph_dict[key][subkey][k].nodes(data=True)), orient='index')
            modstat_dict[key][subkey][k]['group']=subkey


MZ
no
0
no
1
no
2
no
3
no
4
no
5
no
6
no
7
no
ov
0
ov
1
ov
2
ov
3
ov
4
ov
5
ov
6
ov
7
ov
8
ov
ob
0
ob
1
ob
2
ob
3
ob
4
ob
5
ob
6
ob
7
ob
8
ob
DZ
no
0
no
1
no
2
no
3
no
4
no
5
no
ov
0
ov
1
ov
2
ov
3
ov
4
ov
5
ov
6
ov
7
ov
ob
0
ob
1
ob
2
ob
3
ob
4
ob
5
ob
6
ob
7
ob
NR
no
0
no
1
no
2
no
3
no
4
no
5
no
6
no
7
no
ov
0
ov
1
ov
2
ov
3
ov
4
ov
5
ov
6
ov
7
ov
8
ov
9
ov
ob
0
ob
1
ob
2
ob
3
ob
4
ob
5
ob
6
ob
7
ob
8
ob


In [75]:
list(modstat_dict['NR']['ov'].values())

[    centrality  clustering        PC  modules group
 0     0.008991    0.605844 -0.627084        0    ov
 24    0.011428    0.592655  0.443209        0    ov
 38    0.005832    0.562821  0.018082        0    ov
 31    0.010614    0.607381  0.432075        0    ov,
     centrality  clustering          PC  modules group
 1     0.003417    0.729610  -44.001407        1    ov
 35    0.004840    0.636975   -6.041723        1    ov
 36    0.007705    0.483131   -1.434467        1    ov
 5     0.005004    0.557932   -3.313664        1    ov
 8     0.003861    0.711020  -55.644869        1    ov
 9     0.010131    0.485366  -38.915057        1    ov
 41    0.006311    0.678520   -8.984764        1    ov
 11    0.003440    0.703231   -2.473093        1    ov
 43    0.003395    0.655462 -166.050583        1    ov
 72    0.004368    0.706122   -6.181872        1    ov
 21    0.005730    0.659903  -95.888069        1    ov
 59    0.003977    0.575499   -1.268437        1    ov
 28    0.007383    

In [76]:
pandas_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }
for key, item in modstat_dict.items():
    print(key)
    for k, i in item.items():
        print(i)
        pandas_dict[key][k]=pd.concat(list(modstat_dict['NR'][k].values()))
        

MZ
{0:     centrality  clustering        PC  modules group
0     0.008601    0.623027 -0.524893        0    no
24    0.009632    0.609001  0.556957        0    no
38    0.006913    0.535941  0.337075        0    no
31    0.011434    0.596721  0.561884        0    no, 1:     centrality  clustering           PC  modules group
1     0.003276    0.728283  -423.817362        1    no
5     0.004317    0.578629    -6.177036        1    no
6     0.005822    0.627451    -2.640254        1    no
8     0.004574    0.667677 -3959.985355        1    no
9     0.007225    0.514286   -17.489375        1    no
11    0.003974    0.678076    -8.206111        1    no
13    0.004486    0.576102    -3.940682        1    no
14    0.005313    0.652211   -44.522089        1    no
21    0.006557    0.630897 -1274.671038        1    no
23    0.002811    0.677463    -4.325871        1    no
32    0.003887    0.631707    -6.416971        1    no
35    0.003146    0.659091    -3.521804        1    no
39    0.003408

In [77]:
pandas_dict['NR']

{'no':     centrality  clustering          PC  modules group
 0     0.006490    0.643771   -1.032697        0    no
 24    0.012522    0.607639    0.473302        0    no
 38    0.006565    0.547561    0.164628        0    no
 31    0.011106    0.610169    0.543334        0    no
 1     0.002974    0.746860 -136.687992        1    no
 ..         ...         ...         ...      ...   ...
 79    0.003211    0.742836   -4.341724        6    no
 97    0.002353    0.581538 -609.623624        7    no
 90    0.002650    0.734505   -5.212507        7    no
 85    0.004103    0.699565   -2.903856        7    no
 94    0.003176    0.710453   -5.279551        7    no
 
 [100 rows x 5 columns],
 'ov':     centrality  clustering         PC  modules group
 0     0.008991    0.605844  -0.627084        0    ov
 24    0.011428    0.592655   0.443209        0    ov
 38    0.005832    0.562821   0.018082        0    ov
 31    0.010614    0.607381   0.432075        0    ov
 1     0.003417    0.729610 -44

In [78]:
an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',pandas_dict,'8_modulemetrics_dict')

already have tmp


In [ ]:
# test=an.onetoughjar(os.path.join(basepath,'tmp','8_modulemetrics_dict_11-20-2019_10-40-13'))

In [79]:
df=pd.concat(list(pandas_dict['NR'].values()))

In [80]:
df['group'].value_counts()

no    100
ov    100
ob    100
Name: group, dtype: int64

In [81]:
df['group'] = df['group'].astype('category')
df['group'].cat.reorder_categories(['no', 'ov','ob'], inplace=True)
df['modules'] = df['modules'].astype('category')

In [ ]:
# basepath = '/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/'
# file_dict=an.onetoughjar(os.path.join(basepath,'tmp','5_summary_dict_07-31-2020_12-26-08'))

In [109]:
pandas_dict['NR']['no'].keys()

Index(['centrality', 'clustering', 'PC', 'modules', 'group'], dtype='object')

In [106]:
def zed(dicti):
    tmp_dict={}
    for key, value in dicti.items():
        print(key)
        for k, v in value['modules'].items():
            print(k)

In [107]:
zed(pandas_dict['NR'])

no
0
24
38
31
1
5
6
8
9
11
13
14
21
23
28
32
35
36
39
41
43
45
47
49
50
51
52
53
55
56
59
60
63
65
72
2
3
7
10
12
15
16
18
19
20
22
26
27
29
33
34
37
40
42
44
46
80
87
95
96
4
69
71
17
30
57
25
67
73
74
76
77
48
81
82
83
84
54
88
89
58
91
92
61
64
66
98
68
99
75
78
86
62
93
70
79
97
90
85
94
ov
0
24
38
31
1
35
36
5
8
9
41
11
43
72
21
59
28
2
3
7
10
12
15
16
18
19
20
22
26
27
29
33
34
37
40
42
44
46
80
95
96
4
69
71
74
17
82
87
30
57
61
25
32
6
39
13
23
64
66
68
75
14
78
86
62
63
65
67
73
92
45
47
48
49
50
51
52
53
54
55
56
89
58
60
70
76
77
81
83
84
88
91
93
97
98
99
79
85
90
94
ob
0
24
38
31
1
65
67
58
8
41
73
11
45
47
49
50
51
52
21
53
54
55
56
60
2
3
7
10
12
15
16
18
19
20
26
27
29
34
37
40
42
44
46
80
95
96
33
4
69
71
74
48
17
82
22
87
25
61
30
57
35
5
72
9
14
59
28
63
32
36
6
39
43
13
23
64
66
68
75
78
86
62
70
76
77
81
83
84
88
89
91
92
93
97
98
99
79
85
90
94


In [97]:
def zed(dicti):
    tmp_dict={}
    for key, item in dicti.items():
        print(key)
        for subkey, value in item.items():
            print(subkey)
            if subkey in ['modules']:
                for x, y in value.items():
#                     print(x)
                    if x == 'partition':
                        df=pd.DataFrame.from_dict(y, orient='index')
                        df.rename({0: 'module'}, axis=1, inplace = True)
                        df['group']=key
                    if x == 'zdegree':
                        df['zdegree']=y
                        tmp_dict[key]=df
                    else:
                        continue
    total_df=pd.concat(list(tmp_dict.values()))
    return(total_df)



In [98]:
df_z=zed(pandas_dict['NR'])

no
centrality
clustering
PC
modules
group
ov
centrality
clustering
PC
modules
group
ob
centrality
clustering
PC
modules
group


ValueError: No objects to concatenate

In [94]:
df_z

NameError: name 'df_z' is not defined

In [ ]:
df.columns

In [ ]:
df_result = pd.merge(df, df_z, left_index=True, right_index=True)

In [ ]:
df_result

In [ ]:
df.to_csv(os.path.join(basepath, 'tmp', 'mod_data.csv'))

## Summary
No differences in the graph metrics between modules between groups

In [ ]:
subgraph_dict['NR']['no'].keys()

In [ ]:
subgraph_dict['NR']['no']

In [ ]:
edgestat_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }
for key, value in subgraph_dict['NR'].items():
    print(key)        
    liist=list(subgraph_dict['NR'][key].keys())
    for i in liist:
        modstat_dict['NR'][key].update({i:[]})
    for k,v in subval.items():
        print(k)
        print(subkey)
        edgestat_dict['NR'][key][k]=nx.to_numpy_matrix(v)



In [ ]:
edgestat_dict['NR']

In [ ]:
an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',edgestat_dict,'9_edgestat_dict')

In [ ]:
edgestat_dict['NR']['ov']

In [ ]:
edgestat_dict['NR']['no'].keys()

In [ ]:
for key, value in edgestat_dict['NR'].items():
    for k,v in value.items():
        print("This is the %s for the %s module"%(key,k))
#         print(edgestat_dict['NR'][key][k])
        corr_mod(edgestat_dict['NR'][key][k])

In [ ]:
def corrector(x, alpha):
    sh=x.shape
    print(sh)
    results=x[1].ravel()
    mask = np.isfinite(results)
    pval_corrected = np.empty(results.shape)
    pval_corrected.fill(np.nan)
    pval_corrected[mask] = st.stats.multitest.multipletests(results[mask],alpha=alpha,method='fdr_bh')[1]
    p=np.reshape(pval_corrected, sh)
    print(np.nanmin(p))
    ps = 1-p
    print(np.nanmax(ps))
    coor_fig(ps)
    return(p)

def coor_fig(df):
    plt.figure(figsize=(40,25))
    m=np.nanmin(df)
    M=np.nanmax(df)
    print('The max p value is %f'%M)
    sns.heatmap(df, linewidth=0.5, 
                vmin=m, vmax=1, 
                cmap=sns.cubehelix_palette(10000), 
                cbar_kws={'ticks': [0.0, 0.2, 0.4, 0.5, 0.7, 0.8,0.975 ,1.0]})
    return(plt.show())


In [ ]:
def compar(di,mod):
    a=di['NR']['no'][mod]
    print(a)
    b=di['NR']['ov'][mod]
    c=di['NR']['ob'][mod]
    test=spy_stats.f_oneway(a, b, c)
    print(test[1])
    p=corrector(test[1], 0.05)
#     coor_fig(p)

In [ ]:
di=edgestat_dict
mod=0
a=di['NR']['no'][mod]
b=di['NR']['ov'][mod]
c=di['NR']['ob'][mod]
test=spy_stats.f_oneway(a, b, c)

In [ ]:
c

In [ ]:
no_ob=spy_stats.ttest_ind(a, c, nan_policy='omit')
no_ob_results=corrector(no_ob[1], 0.025)

In [ ]:
compar(edgestat_dict, 0)

### Between no and ov & no and ob
p=0.05/2  
p=0.025

In [ ]:
no_ov=spy_stats.ttest_ind(a, b, nan_policy='omit')
no_ov_results=corrector(no_ov, 0.025)

In [ ]:
no_ob=spy_stats.ttest_ind(a, c, nan_policy='omit')
no_ob_results=corrector(no_ob, 0.025)

# Within module metrics

# Edge analysis

In [ ]:
save_dict=an.onetoughjar(os.path.join(basepath,'tmp','save_dict_11-14-2019_02-46-01'))

In [ ]:
save_dict['NR']['no'].keys()

In [ ]:
for key, item in save_dict['NR'].items():
    for subkey, value in item.items():
        print(subkey)
        if subkey in ['mean_FC']:
            tmp=pd.DataFrame.from_dict(value, orient='index')
            tmp['group'] = '%s'%key
            tmp.to_csv(os.path.join(basepath,'tmp','%s_%s_data_per_sub.csv'%(key,subkey)), sep=',')


# Using visbrain to look at the cifti files

In [ ]:
# Scene creation
sc = SceneObj(bgcolor='black', size=(1400, 1000))
# Colorbar default arguments. See `visbrain.objects.ColorbarObj`
CBAR_STATE = dict(cbtxtsz=12, txtsz=10., width=.1, cbtxtsh=3.,
                  rect=(-.3, -2., 1., 4.))
KW = dict(title_size=14., zoom=1.2)


In [ ]:
# Download the annotation file of the left hemisphere lh.aparc.a2009s.annot
path_to_file1 = download_file('lh.aparc.a2009s.annot', astype='example_data')
# Define the brain object (now you should know how to do it)
b_obj_parl = BrainObj('inflated', hemisphere='left', translucent=False)
# Print parcellates included in the file
# print(b_obj_parl.get_parcellates(path_to_file1))
# Finally, parcellize the brain and add the brain to the scene
b_obj_parl.parcellize(path_to_file1)
sc.add_to_subplot(b_obj_parl, row=1, col=1, rotate='left',
                  title='Parcellize using the Desikan Atlas', **KW)


In [ ]:
# Download the lh.sig.nii.gz file
file = download_file('lh.sig.nii.gz', astype='example_data')
# Define the [...] you know
b_obj_fmri = BrainObj('inflated', translucent=False, sulcus=True)
# Add fMRI activation and hide every activation that is under 5.
b_obj_fmri.add_activation(file=file, clim=(5., 20.), hide_under=5,
                          cmap='viridis', hemisphere='left')
sc.add_to_subplot(b_obj_fmri, row=2, col=1, title='Add fMRI activation',
                  rotate='left', **KW)


In [ ]:
# Link the rotation of subplots (row=0, col=1) and (row=1, col=2)
# sc.link((0, 1), (1, 2))
# Screenshot of the scene
# sc.screenshot('ex_brain_obj.png', transparent=True)

sc.preview()